# Расчет результатов

1. Посчитать модели линейной регрессии для 20 зданий по оптимальному набору параметров: метеорологические данные, дни недели, недели года, месяцы и праздники по всему набору данных.
2. Загрузить данные решения, посчитать значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузить результат в файл.